# Module 10: SDOH Data from AHRQ Database

Author: Jackson Viscomi <br>
Class: BMI 6010

## Dataset

In [2]:
import pandas as pd

Dataset Link = https://www.ahrq.gov/sites/default/files/wysiwyg/sdoh/SDOH_2020_Codebook_1_0.xlsx

In [3]:
path = "C:/Users/jcviscom/OneDrive - Intermountain Healthcare/BMI Degree/6018/Module 10/SDOH_Dataset.csv" #change with your local path
SDOH_Data = pd.read_csv(path)

## Melt <br>

In [4]:
#Showing every county in Utah
#dropping duplicates so it's one-row-per-county
SDOH_melt = pd.melt(SDOH_Data[SDOH_Data.STATE == 'Utah'], id_vars = ['STATE'], value_vars = ['COUNTY']).drop_duplicates()
SDOH_melt.head()

,STATE,variable,value
0,Utah,COUNTY,Beaver County
2,Utah,COUNTY,Box Elder County
14,Utah,COUNTY,Cache County
42,Utah,COUNTY,Carbon County
47,Utah,COUNTY,Daggett County


## Group By

In [5]:
#Grouping by each State, County to show total population per county
#Using agg function so that we can rename the population as 'total pop' (will need to be referened next step)
SDOH_GroupBy = SDOH_Data[SDOH_Data.STATE == 'Utah'].groupby(["STATE", "COUNTY"], as_index = False)\
.agg(total_pop = ('ACS_TOT_POP_WT', 'sum'))
#Changing total_pop from float to int, since there aren't fractional people
SDOH_GroupBy['total_pop'] = SDOH_GroupBy['total_pop'].astype(int)
SDOH_GroupBy

,STATE,COUNTY,total_pop
0,Utah,Beaver County,6594
1,Utah,Box Elder County,54953
2,Utah,Cache County,126336
3,Utah,Carbon County,20401
4,Utah,Daggett County,590
5,Utah,Davis County,350761
6,Utah,Duchesne County,19950
7,Utah,Emery County,10099
8,Utah,Garfield County,5000
9,Utah,Grand County,9698


## Pivot<br>

In [6]:
#showing population by country from SDOH_GroupBy in a long format
#using state as index, columns as county, and values as the total population
SDOH_Pivot = SDOH_GroupBy.pivot(index = 'STATE', columns = 'COUNTY', values = 'total_pop')
SDOH_Pivot.head()

COUNTY,Beaver County,Box Elder County,Cache County,Carbon County,Daggett County,Davis County,Duchesne County,Emery County,Garfield County,Grand County,...,Sanpete County,Sevier County,Summit County,Tooele County,Uintah County,Utah County,Wasatch County,Washington County,Wayne County,Weber County
STATE,,,,,,,,,,,,,,,,,,,,,
Utah,6594,54953,126336,20401,590,350761,19950,10099,5000,9698,...,30421,21475,41680,69740,35736,621506,33053,172127,2698,255284


## Aggregation<br>

In [15]:
#Showing % of total population in poverty
#filtering down to utah
#grouping by state, county
SDOH_GroupBy2 =\
SDOH_Data[SDOH_Data.STATE == 'Utah']\
.groupby(["STATE", "COUNTY"], as_index = False)\
.aggregate(total_pop = ('ACS_TOT_POP_WT', 'sum'), tot_pop_poverty = ('ACS_PCT_PERSON_INC_BELOW99','sum')) #two aggregates: total pop, total pop in poverty

SDOH_GroupBy2['PovertyRatio'] = SDOH_GroupBy2['tot_pop_poverty']/SDOH_GroupBy2['total_pop'] #creating a new column that the past two
SDOH_GroupBy2 = SDOH_GroupBy2.sort_values(by = ['PovertyRatio'], ascending = False) #sorting descending by pov ratio so most impoverished counties are first
SDOH_GroupBy2


,STATE,COUNTY,total_pop,tot_pop_poverty,PovertyRatio
15,Utah,Piute County,1870.0,19.13,0.010230
8,Utah,Garfield County,5000.0,33.35,0.006670
18,Utah,San Juan County,15295.0,101.87,0.006660
4,Utah,Daggett County,590.0,3.28,0.005559
16,Utah,Rich County,2415.0,10.46,0.004331
10,Utah,Iron County,53148.0,211.62,0.003982
3,Utah,Carbon County,20401.0,78.97,0.003871
27,Utah,Wayne County,2698.0,10.15,0.003762
7,Utah,Emery County,10099.0,36.78,0.003642
12,Utah,Kane County,7658.0,27.62,0.003607


## Iteration<br>

In [16]:
SDOH_Iterrow = SDOH_Data[['COUNTY','STATE']][SDOH_Data.STATE == 'Utah'].drop_duplicates() #showing unique counties in Utah

for index, row in SDOH_Iterrow.iterrows():  #iterating over each item in the county index using .iterrows()
    print(row["COUNTY"])

Beaver County
Box Elder County
Cache County
Carbon County
Daggett County
Davis County
Duchesne County
Emery County
Garfield County
Grand County
Iron County
Juab County
Kane County
Millard County
Morgan County
Piute County
Rich County
Salt Lake County
San Juan County
Sanpete County
Sevier County
Summit County
Tooele County
Uintah County
Utah County
Wasatch County
Washington County
Wayne County
Weber County


In [17]:
#iterating over each state index label and value using .iteritems()
for items in SDOH_Data['STATE'].drop_duplicates().iteritems():
    print(items)

(0, 'Alabama')
(1437, 'Alaska')
(1614, 'Arizona')
(3379, 'Arkansas')
(4202, 'California')
(13331, 'Colorado')
(14778, 'Connecticut')
(15661, 'Delaware')
(15923, 'District of Columbia')
(16129, 'Florida')
(21289, 'Georgia')
(24085, 'Hawaii')
(24546, 'Idaho')
(25002, 'Illinois')
(28267, 'Indiana')
(29963, 'Iowa')
(30859, 'Kansas')
(31688, 'Kentucky')
(32994, 'Louisiana')
(34382, 'Maine')
(34789, 'Maryland')
(36264, 'Massachusetts')
(37884, 'Michigan')
(40901, 'Minnesota')
(42406, 'Mississippi')
(43284, 'Missouri')
(44938, 'Montana')
(45257, 'Nebraska')
(45810, 'Nevada')
(46589, 'New Hampshire')
(46939, 'New Jersey')
(49120, 'New Mexico')
(49732, 'New York')
(55143, 'North Carolina')
(57815, 'North Dakota')
(58043, 'Ohio')
(61211, 'Oklahoma')
(62416, 'Oregon')
(63417, 'Pennsylvania')
(66863, 'Rhode Island')
(67113, 'South Carolina')
(68436, 'South Dakota')
(68678, 'Tennessee')
(70379, 'Texas')
(77275, 'Utah')
(77991, 'Vermont')
(78184, 'Virginia')
(80382, 'Washington')
(82166, 'West Virgi